<a href="https://colab.research.google.com/github/wingated/cs473/blob/main/labs/cs473_lab_week_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><p><b>After clicking the "Open in Colab" link, copy the notebook to your own Google Drive before getting started, or it will not save your work</b></p>

# BYU CS 473 Lab Week 6

## Introduction:

The concept of mutual information is powerful in theory, but in practice it can be difficult to estimate. In this lab, you will implement a conceptually simple way to estimate the mutual information between two random variables. It is known as the KSG estimator (from "Kraskov–Stögbauer–Grassberger").

You will then use your estimator as part of a feature selection algorithm.

---
## Grading standards   

Your notebook will be graded on the following:

* 60% Correct implementation of KSG estimator
* 20% Correct implementation of "all_mutual_inf"
* 20% Correct selection of most informative columns

---
## Description

For this lab, we will use this dataset [lab_week_6_data.csv](https://github.com/wingated/cs473/blob/main/labs/data/lab_week_6_data.csv).

Download the csv from the link above, then upload the csv to your current google colab runtime.

Then begin by loading and preparing the dataset:

In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv( "lab_week_6_data.csv" )

X = df[['x1','x2','x3','x4','x5', 'x6', 'x7']]
y = df['y']

X = np.array(X)
y = np.array(y)


The KSG estimator is an elegant idea that's fairly easy to implement. The idea is to use an adaptive grid, where the grid depends on the local density of the data.  This is accomplished by calculating the k'th nearest neighbor for each point, and then defining a box around each point that just barely extends to include the k'th nearest neighbor. Importantly, the size of the box is also used in the marginal entropy calculations

To get some intuition for the method, check out [this demonstration on Wolfram Alpha's site](https://demonstrations.wolfram.com/KraskovKSGEstimatorOfMutualInformation/).  Here is a screenshot of their UI:

![Wolfram UI](https://raw.githubusercontent.com/wingated/cs473/main/labs/images/lab_week_6_image1.png)

The algorithm is fairly straightforward.  We are given N samples from the joint distribution of p(X,Y). First, we calculate the *local mutual information* for each data point:

1. For each datapoint $d_j = (x_j,y_j)$
2. Find the k'th nearest neighbor (call this point $n_j$) -- you should use the infinity norm (max norm), and distance should be calculated using both coordinates (ie, don't just calculate distances using only the x or y coordinate)
3. Count the number of points in x whose 'x-distance' fall within the distance to the k'th nearest neighbor (calculated above) of $d_j$ (ie, ignore y); call this number $n_{x{_j}}$.  Do the same for the points in y (these are the shaded gray boxes in the image); call it $n_{y{_j}}$.

Then, we calculate the final mutual information by averaging over all of the datapoints and adding a few more digamma calls:

![final mutual information](https://raw.githubusercontent.com/wingated/cs473/main/labs/images/lab_week_6_image3.png)

where $\Psi$ is the digamma function

If you want more details on the implementation, see page 2 of [Estimating Mutual Information](https://arxiv.org/pdf/cond-mat/0305641)

In [23]:
from scipy.special import digamma


def mutual_inf(X, Y, k=3):
    # X and Y are lists of length n of samples from p(X,Y)
    # this function returns a single scalar

    n = len(X)
    mi = 0.0

    # Compare every row of data to every other row
    for j in range(n):
        k_smallest = []
        x_diffs = []
        y_diffs = []

        for i in range(n):
            if j == i:
                continue

            # Find the largest distance between X-values and Y-values
            x_diffs.append(abs(X[j] - X[i]))
            y_diffs.append(abs(Y[j] - Y[i]))

            # Find the overall max distance
            max_diff = max(x_diffs[-1], y_diffs[-1])

            # Save the k smallest max distances between data[j] and all other rows
            if len(k_smallest) < k:
                k_smallest.append(max_diff)

            else:
                for index in range(len(k_smallest)):
                    if max_diff < k_smallest[index]:
                        temp = k_smallest[index]
                        k_smallest[index] = max_diff
                        max_diff = temp

        # at this point, k_smallest contains the k smallest distances for point j
        # the k-th nearest neighbor distance is k_smallest[-1]


        # Count how many times the largest distance between X-values and Y-values is less than the k-th smallest distance
        n_xj = sum(1 for d in x_diffs if d <= k_smallest[-1])
        n_yj = sum(1 for d in y_diffs if d <= k_smallest[-1])

        mi += digamma(k) - digamma(n_xj + 1) - digamma(n_yj + 1) + digamma(n)

    return mi / n

---
## Part 2:

Now that you can estimate mutual information between two variables, we're going to use your method in the context of a regression problem.

Given our X,y data from the beginning of the lab, run your function and calculate the mutual information between every column of X and y.  This should result in a list of scores, that might look something like this:

```python
array([1.2641203602210282, 0.07974959311825458, 0.03321433628330617, 0.02213361087954091, 0.0, 0.0, 0.0])
```

If you wrap this in a single function (call it "all_mutual_inf"), then you can use scikit learn's feature selection algorithms:

In [24]:
def all_mutual_inf( X, y, k = 3 ):
    # return a list of mutual informations
    # the list should be as long as the number of columns in X

    mi_scores = []
    for i in range(X.shape[1]):  # Iterate over each feature (column) in X
        mi_score = mutual_inf(X[:, i], y, k)  # Compute MI for the i-th feature
        mi_scores.append(mi_score)

    # SelectKBest expects two outputs: scores and p-values.
    # Since we don't have p-values for this MI estimator,
    # return a dummy array of zeros for the p-values.
    return mi_scores, np.zeros(len(mi_scores))

In [30]:
from sklearn.feature_selection import SelectKBest

selector = SelectKBest(all_mutual_inf, k=2)
X_new = selector.fit_transform(X, y)
top_features = selector.get_support(indices=True)

print(X_new)
print("\nMost informative features:", top_features)

[[ 0.00000000e+00 -4.24611223e+00]
 [ 1.01010101e-01 -1.18112789e+00]
 [ 2.02020202e-01 -8.66527388e-01]
 [ 3.03030303e-01 -2.16440757e+00]
 [ 4.04040404e-01 -1.60624812e-01]
 [ 5.05050505e-01  1.61619297e+00]
 [ 6.06060606e-01  6.14340619e+00]
 [ 7.07070707e-01  1.08939000e+00]
 [ 8.08080808e-01  1.41911582e+00]
 [ 9.09090909e-01  5.03934980e-01]
 [ 1.01010101e+00 -4.94823284e+00]
 [ 1.11111111e+00  8.09347263e-01]
 [ 1.21212121e+00  1.15038760e+00]
 [ 1.31313131e+00  8.44023139e+00]
 [ 1.41414141e+00  5.54230237e-01]
 [ 1.51515152e+00  2.11676324e+00]
 [ 1.61616162e+00  1.18879398e+00]
 [ 1.71717172e+00 -2.13229674e+00]
 [ 1.81818182e+00  4.88301390e+00]
 [ 1.91919192e+00  3.79115263e+00]
 [ 2.02020202e+00  3.98925746e+00]
 [ 2.12121212e+00 -1.03119267e+00]
 [ 2.22222222e+00  5.98616071e+00]
 [ 2.32323232e+00 -2.34696733e+00]
 [ 2.42424242e+00  3.69996522e+00]
 [ 2.52525253e+00  8.59156890e+00]
 [ 2.62626263e+00 -8.70598874e-01]
 [ 2.72727273e+00  4.82924993e-01]
 [ 2.82828283e+00  2

So: which X features are most informative about y?

Most informative features: [0 1]

---
## Hints

The following functions may be useful to you: scipy.special.digamma

For fun, you could also [read more about the KSG estimator](https://arxiv.org/pdf/1604.03006).